-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

d
# Views and CTEs on Databricks
In this demonstration, you will create and explore views and common table expressions (CTEs).

## Learning Objectives
By the end of this lesson, you will be able to:
* Use Spark SQL DDL to define views
* Run queries that use common table expressions



**Resources**
* [Create View - Databricks Docs](https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-view.html)
* [Common Table Expressions - Databricks Docs](https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-qry-select-cte.html)

## Classroom Setup
The following cell is for setting up the classroom. It simply installs a python library that is used to generate variables, configure a temporary directory, and import a dataset we will use later in the lesson.

In [0]:
%python
import sys, subprocess, os
subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/databricks-academy/user-setup"])

from dbacademy import LessonConfig
LessonConfig.configure(course_name="Databases Tables and Views on Databricks", use_db=False)
LessonConfig.install_datasets(silent=True)

In [0]:
%python 
dbutils.widgets.text("username", LessonConfig.clean_username)
dbutils.widgets.text("working_directory", LessonConfig.working_dir)

## Important Note
In order to keep from conflicting with other users and to ensure the code below runs correctly, there are places in the code that use widgets to store and use variables (e.g., `${username}`). You should not have to change these in order to make the code work correctly.

We start by creating a table of data we can use for the demonstration.

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS ${username}_training_database;
USE ${username}_training_database;

-- mode "FAILFAST" will abort file parsing with a RuntimeException if any malformed lines are encountered
CREATE OR REPLACE TEMPORARY VIEW temp_delays USING CSV OPTIONS (
  path '${working_directory}/datasets/flights/departuredelays.csv',
  header "true",
  mode "FAILFAST"
);
CREATE OR REPLACE TABLE external_table LOCATION '${working_directory}/external_table' AS
  SELECT * FROM temp_delays;

SELECT * FROM external_table;

date,delay,distance,origin,destination
01011245,6,602,ABE,ATL
01020600,-8,369,ABE,DTW
01021245,-2,602,ABE,ATL
01020605,-4,602,ABE,ATL
01031245,-4,602,ABE,ATL
01030605,0,602,ABE,ATL
01041243,10,602,ABE,ATL
01040605,28,602,ABE,ATL
01051245,88,602,ABE,ATL
01050605,9,602,ABE,ATL


## Views
Let's create a view that contains only the data where the origin is 'ABQ' and the destination is 'LAX'.

In [0]:
%sql
CREATE OR REPLACE VIEW view_delays_ABQ_LAX AS
SELECT * FROM external_table WHERE origin = 'ABQ' AND destination = 'LAX';
SELECT * FROM view_delays_ABQ_LAX;

date,delay,distance,origin,destination
03010545,-2,589,ABQ,LAX
03021950,105,589,ABQ,LAX
03031950,15,589,ABQ,LAX
03030545,294,589,ABQ,LAX
03041950,-8,589,ABQ,LAX
03040545,11,589,ABQ,LAX
03051950,27,589,ABQ,LAX
03050545,-5,589,ABQ,LAX
03061930,6,589,ABQ,LAX
03060620,2,589,ABQ,LAX


To show a list of tables (and views), we use the `SHOW TABLES` command.  
  
Note that the `view_delays_abq_lax` view is in the list. If we detach from, and reattach to, the cluster and reload the list of tables, view_delays_abq_lax persists. This is because View metadata (name, location, etc.) are stored in the metastore.

(The command `USE ${username}_training_database;` is used after reattaching to the cluster because state is lost when the SparkSession is deleted)

In [0]:
%sql
USE ${username}_training_database;
SHOW tables;

database,tableName,isTemporary
jaime_vera_palomino_gmail_com_training_database,external_table,false
jaime_vera_palomino_gmail_com_training_database,view_delays_abq_lax,false
,temp_delays,true


Now, let's create a temporary view. The syntax is very similar but adds `TEMPORARY` to the command.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW temp_view AS
SELECT * FROM external_table WHERE delay > 120 ORDER BY delay ASC;
SELECT * FROM temp_view;

date,delay,distance,origin,destination
01161810,1000,887,IND,MIA
02152001,1003,1413,JFK,DEN
01311800,1004,3288,HNL,DFW
03070600,1004,287,FAY,ATL
03061605,1004,2221,HNL,LAX
03021925,1006,355,JAN,DFW
03310645,1006,974,RSW,ORD
03142131,1007,2189,ANC,MSP
03121647,1009,482,CLE,ATL
02051715,1009,602,OKC,ORD


Let's again show list of tables (and views).  
  
Two things we note are that the `temp_view` view is in the list and that `temp_view` is marked `isTemporary`.  
  
If we detach from, and reattach to, the cluster and reload the list of tables, `temp_view` is deleted. This is because temporary view metadata (name, location, etc.) are not stored in the metastore. When we detach from the cluster, the Spark session is deleted, which deletes the temporary view.

In [0]:
%sql
USE ${username}_training_database;
SHOW TABLES;

database,tableName,isTemporary
jaime_vera_palomino_gmail_com_training_database,external_table,false
jaime_vera_palomino_gmail_com_training_database,view_delays_abq_lax,false
,temp_delays,true
,temp_view,true


Let's now create a global temporary view. We add `GLOBAL` to the command. This view is just like the temporary view above, but it is different in one important way. It is added to the `global_temp` database that exists on the cluster. As long as the cluster is running, this database persists, and any notebooks attached to the cluster can access its global temporary views.  
  
Note when we use global temporary views, we have to prefix them with `global_temp.` since we are accessing the `global_temp` database.

In [0]:
%sql
CREATE OR REPLACE GLOBAL TEMPORARY VIEW global_temp_view_distance AS
SELECT * FROM external_table WHERE distance > 1000;
SELECT * FROM global_temp.global_temp_view_distance;

date,delay,distance,origin,destination
02151303,16,1516,ORD,LAX
02151157,7,1316,ORD,LAS
02151818,55,1511,ORD,PDX
02150941,0,1499,ORD,SNA
02151320,17,1604,ORD,SFO
02151804,2,1497,ORD,SAN
02150947,-1,1316,ORD,LAS
02152030,24,1604,ORD,SFO
02150945,19,1497,ORD,SAN
02150853,30,1838,ORD,STT


Again, global temporary views are available to any notebook attached to the cluster, including
- New notebooks
- This notebook, even if it is detached from, and reattached to, the cluster

In [0]:
%sql
SELECT * FROM global_temp.global_temp_view_distance;

date,delay,distance,origin,destination
02151303,16,1516,ORD,LAX
02151157,7,1316,ORD,LAS
02151818,55,1511,ORD,PDX
02150941,0,1499,ORD,SNA
02151320,17,1604,ORD,SFO
02151804,2,1497,ORD,SAN
02150947,-1,1316,ORD,LAS
02152030,24,1604,ORD,SFO
02150945,19,1497,ORD,SAN
02150853,30,1838,ORD,STT


One thing to note is that global temporary views do not show in the list of tables.

In [0]:
%sql
USE ${username}_training_database;
SHOW TABLES;

database,tableName,isTemporary
jaime_vera_palomino_gmail_com_training_database,external_table,false
jaime_vera_palomino_gmail_com_training_database,view_delays_abq_lax,false
,temp_delays,true
,temp_view,true


## Common Table Expressions (CTEs)
CTEs can be used in a variety of contexts. Below, are a few examples of the different ways a CTE can be used in a query. First, an example of making multiple column aliases using a CTE.

In [0]:
%sql
WITH flight_delays(
  total_delay_time,
  origin_airport,
  destination_airport
) AS (
  SELECT
    delay,
    origin,
    destination
  FROM
    external_table
)
SELECT
  *
FROM
  flight_delays
WHERE
  total_delay_time > 120
  AND origin_airport = "ATL"
  AND destination_airport = "DEN";

total_delay_time,origin_airport,destination_airport
173,ATL,DEN
176,ATL,DEN
189,ATL,DEN
135,ATL,DEN
155,ATL,DEN
333,ATL,DEN
157,ATL,DEN
172,ATL,DEN
128,ATL,DEN
178,ATL,DEN


Next, is an example of a CTE in a CTE definition.

In [0]:
%sql
WITH lax_bos AS (
  WITH origin_destination (origin_airport, destination_airport) AS (
    SELECT
      origin,
      destination
    from
      external_table
  )
  SELECT
    *
  FROM
    origin_destination
  WHERE
    origin_airport = 'LAX'
    AND destination_airport = 'BOS'
)
SELECT
  count(origin_airport) AS `Total Flights from LAX to BOS`
FROM
  lax_bos;

Total Flights from LAX to BOS
684


Now, here is an example of a CTE in a subquery.

In [0]:
%sql
SELECT
  max(total_delay) AS `Longest Delay (in minutes)`
FROM
  (
    WITH delayed_flights(total_delay) AS (
      SELECT
        delay
      from
        external_table
    )
    SELECT
      *
    FROM
      delayed_flights
  );

Longest Delay (in minutes)
995


We can also use a CTE in a subquery expression.

In [0]:
%sql
SELECT
  (
    WITH distinct_origins AS (
      SELECT DISTINCT origin FROM external_table
    )
    SELECT
      count(origin)
    FROM
      distinct_origins
  ) AS `Number of Different Origin Airports`;

Number of Different Origin Airports
255


Finally, here is a CTE in a CREATE VIEW statement.

In [0]:
%sql
CREATE OR REPLACE VIEW BOS_LAX AS
WITH origin_destination(origin_airport, destination_airport) AS 
(SELECT origin, destination FROM external_table)
SELECT * FROM origin_destination
WHERE origin_airport = 'BOS' AND destination_airport = 'LAX';
SELECT count(origin_airport) AS `Number of Delayed Flights from BOS to LAX` FROM BOS_LAX;

Number of Delayed Flights from BOS to LAX
677


## Clean up 
We first drop the training database.

In [0]:
%sql
DROP DATABASE ${username}_training_database CASCADE;

Finally, we delete the working directory.

In [0]:
%python 
path = dbutils.widgets.get("working_directory")
dbutils.fs.rm(path, True)

Out[1]: True

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>